In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold,train_test_split
import sys
import os
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
##TODO
##figure out the correct way for importing
sys.path.append('..')
#sys.path.append('src')
#print(sys.path)
#from config import config
from model.analyzers import IA_get_model,get_bayes_cv_tuner,GroupKFoldWrapper
from config import config
from NER.ner import load_ner, ner_prediction

In [2]:
def process_NER_LABEL(x):
    if x is None:
        return 'None'
    if pd.isnull(x):
        return 'None'
    else:
        return x

def label_encode(y):
    # I am aware,I just don't want to use label encoder in sklearn
    out = y.copy()
    out[y == 'payroll'] = 1
    out[y == 'other'] = 0
    out[y == 'transfer'] = 3
    out[y == 'loan'] = 2
    return out

#get the model
model = IA_get_model(config,load_model = False, training = True)


In [3]:
X = pd.read_csv("../training_data/intermediate_training_step/augmented_training.csv")
X = X.drop(columns = ['index'])
y = X.loc[:,'handlabel_cat']

In [4]:
X_cred = X[X.debit_only!=1]
y_cred = X_cred['handlabel_cat']
y_cred = label_encode(y_cred).astype(int)

# train_idx,test_idx = train_test_split(X_cred.index,test_size = 0.2,random_state = 1129)
# X_cred_train = X_cred.loc[train_idx]
# X_cred_test = X_cred.loc[test_idx]
# y_cred_train = y_cred.loc[train_idx]
# y_cred_test = y_cred.loc[test_idx]

In [5]:
cv_obj = GroupKFoldWrapper(GroupKFold(config.IA_N_FOLD),X_cred.accountGuid)
#fit_params = {"sample_weight":sample_weights}
#bayes_cv_tuner = get_bayes_cv_tuner(model[2:],cv_obj,config,fit_params)
bayes_cv_tuner = get_bayes_cv_tuner(model[2:],cv_obj,config,{})
np.int = int
clf = bayes_cv_tuner.fit(X_cred,y_cred)
opt_clf = clf.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START xgb__colsample_bylevel=0.7654820824760737, xgb__colsample_bytree=0.8380397544384568, xgb__learning_rate=0.17596224946003122, xgb__max_delta_step=13, xgb__max_depth=5, xgb__min_child_weight=10, xgb__n_estimators=7, xgb__reg_alpha=1.0345051245753473e-09, xgb__subsample=0.7930175461090434
[CV 1/5; 1/1] END xgb__colsample_bylevel=0.7654820824760737, xgb__colsample_bytree=0.8380397544384568, xgb__learning_rate=0.17596224946003122, xgb__max_delta_step=13, xgb__max_depth=5, xgb__min_child_weight=10, xgb__n_estimators=7, xgb__reg_alpha=1.0345051245753473e-09, xgb__subsample=0.7930175461090434;, score=0.874 total time=   4.6s
[CV 2/5; 1/1] START xgb__colsample_bylevel=0.7654820824760737, xgb__colsample_bytree=0.8380397544384568, xgb__learning_rate=0.17596224946003122, xgb__max_delta_step=13, xgb__max_depth=5, xgb__min_child_weight=10, xgb__n_estimators=7, xgb__reg_alpha=1.0345051245753473e-09, xgb__subsample=0.793017

In [6]:
cvr_who,cvr_how,cvr_why,xgb = clf.best_estimator_['cvr_WHO'],clf.best_estimator_['cvr_HOW'], clf.best_estimator_['cvr_WHY'],clf.best_estimator_['xgb']

import joblib
joblib.dump([cvr_who,cvr_how,cvr_why,xgb],'multi_cat_model_partial.pkl')

['multi_cat_model_partial.pkl']

In [7]:
importance_dict = pd.DataFrame({"column_name": clf.best_estimator_['xgb'].get_booster(
).feature_names, "importance": clf.best_estimator_['xgb'].feature_importances_})
importance_dict = importance_dict[importance_dict.importance >
                                  0].sort_values("importance", ascending=False)

In [9]:
importance_dict

,column_name,importance
7,credit_time_interval_1_percentage,0.139569
20,has_strong_payroll_indicators,0.099172
97,n_gram_HOW_zelle,0.098347
33,payment_amount_max,0.095943
21,has_strong_transfer_indicators,0.085749
...,...,...
41,credit_t1_6-8,0.000339
49,credit_t2_6-8,0.000266
139,n_gram_WHY_deposit,0.000074
67,debit_t2_13-17,0.000071


In [8]:
clf.best_score_

0.8992288833107495